In [1]:
%pylab inline

from __future__ import (division, print_function)

import os
import sys
import copy
import fnmatch
import warnings
import collections

import numpy as np
import scipy
try:
    from scipy.stats import scoreatpercentile
except:
    scoreatpercentile = False
from scipy.interpolate import interp1d
import cPickle as pickle

# Astropy
from astropy.io import fits
from astropy    import units as u
from astropy.stats import sigma_clip
from astropy.table import Table, Column
from astropy.utils.console import ProgressBar

# AstroML
from astroML.plotting import hist

# Matplotlib related
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from matplotlib.ticker import NullFormatter
from matplotlib.ticker import MaxNLocator
# Matplotlib default settings
rcdef = plt.rcParams.copy()
pylab.rcParams['figure.figsize'] = 12, 10
pylab.rcParams['xtick.major.size'] = 8.0
pylab.rcParams['xtick.major.width'] = 2.5
pylab.rcParams['xtick.minor.size'] = 4.0
pylab.rcParams['xtick.minor.width'] = 2.5
pylab.rcParams['ytick.major.size'] = 8.0
pylab.rcParams['ytick.major.width'] = 2.5
pylab.rcParams['ytick.minor.size'] = 4.0
pylab.rcParams['ytick.minor.width'] = 2.5

# Personal
import hscUtils as hUtil
import galSBP

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Absolute magnitude of sun in HSC filters

# Actuall borrowed from DES filters
# Values from magsun.data in FSPS
amag_sun_des_g = 5.08
amag_sun_des_r = 4.62
amag_sun_des_i = 4.52
amag_sun_des_z = 4.52
amag_sun_des_y = 4.51

# Based on http://www.baryons.org/ezgal/filters.php
amag_sun_ukiss_y = 4.515

# Extinction correction factor for HSC 
## A\_lambda = Coeff * E(B-V) 

a_hsc_g = 3.233
a_hsc_r = 2.291 
a_hsc_i = 1.635
a_hsc_z = 1.261
a_hsc_y = 1.076

# 
SIGMA1 = 0.3173
SIGMA2 = 0.0455
SIGMA3 = 0.0027

In [3]:
# Code for Get Bootstrap mean or median 
def _confidence_interval_1d(A, alpha=.05, metric=np.mean, numResamples=10000, interpolate=True):
    """Calculates bootstrap confidence interval along one dimensional array"""
    
    if not isinstance(alpha, collections.Iterable):
        alpha = np.array([alpha])

    N = len(A)
    resampleInds = np.random.randint(0, N, (numResamples,N))
    metricOfResampled = metric(A[resampleInds], axis=-1)

    confidenceInterval = np.zeros(2*len(alpha),dtype='float')
    
    if interpolate:
        for thisAlphaInd, thisAlpha in enumerate(alpha):
            confidenceInterval[2*thisAlphaInd] = scoreatpercentile(metricOfResampled, 
                                                                   thisAlpha*100/2.0)
            confidenceInterval[2*thisAlphaInd+1] = scoreatpercentile(metricOfResampled, 
                                                                     100-thisAlpha*100/2.0)
    else:
        sortedMetricOfResampled = np.sort(metricOfResampled)
        for thisAlphaInd, thisAlpha in enumerate(alpha):
            confidenceInterval[2*thisAlphaInd] = sortedMetricOfResampled[int(round(thisAlpha*numResamples/2.0))]
            confidenceInterval[2*thisAlphaInd+1] = sortedMetricOfResampled[int(round(numResamples - 
                                                                                     thisAlpha*numResamples/2.0))]
    return confidenceInterval
    
def _ma_confidence_interval_1d(A, alpha=.05, metric=np.mean, numResamples=10000, interpolate=True):
    A = np.ma.masked_invalid(A, copy=True)
    A = A.compressed()
    confidenceInterval = _confidence_interval_1d(A, alpha, metric, numResamples, interpolate)
    return confidenceInterval

def confidence_interval(A, axis=None, alpha=.05, metric=np.mean, numResamples=10000, interpolate=True):
    """Return the bootstrap confidence interval of an array or along an axis ignoring NaNs and masked elements.
    
    Parameters
    ----------
    A : array_like
        Array containing numbers whose confidence interval is desired. 
    axis : int, optional
        Axis along which the confidence interval is computed.
        The default is to compute the confidence interval of the flattened array.
    alpha: float or array, optional
        confidence level of confidence interval. 100.0*(1-alpha) percent confidence 
        interval will be returned.
        If length-n array, n confidence intervals will be computed
        The default is .05
    metric : numpy function, optional
        metric to calculate confidence interval for.
        The default is numpy.mean
    numResamples : int, optional
        number of bootstrap samples. The default is 10000.
    interpolate: bool, optional
        uses scipy.stats.scoreatpercentile to interpolate between bootstrap samples 
        if alpha*numResamples/2.0 is not integer.
        The default is True
        
    Returns
    -------
    confidenceInterval : ndarray
    An array with the same shape as `A`, with the specified axis replaced by one twice the length of the alpha
    If `A` is a 0-d array, or if axis is None, a length-2 ndarray is returned.
    """
    if interpolate is True and scoreatpercentile is False:
        print("need scipy to interpolate between values")
        interpolate = False
    A = A.copy()
    if axis is None:
        A = A.ravel()
        outA = _ma_confidence_interval_1d(A, alpha, metric, numResamples, interpolate)
    else:
        outA = np.apply_along_axis(_ma_confidence_interval_1d, axis, A, alpha, 
                                   metric, numResamples, interpolate)
        
    return outA

In [4]:
def normProf(sma, sbp, minSma, maxSma): 
    """
    Naive method to normalize the profile. 
    
    Parameters: 
        sbp    : Array for surface brightness profile 
        sma    : Radius range 
        minSma : Minimum SMA
        maxSma   Maximum SMA
    """
    offset = np.nanmedian(sbp[(sma >= minSma) & 
                              (sma <= maxSma)])
    return (sbp-offset)

In [5]:
def pixKpc(redshift, pix=0.168, show=True, npix=1.0):
    """
    Get the corresponding Kpc size of a pixel.  
    
    Parameters: 
    """
    pixKpc = pix * npix * hUtil.cosmoScale(redshift)

    if show:
        print("# %d pixel(s) = %6.3f Kpc" % (npix, pixKpc))
        
    return pixKpc

In [6]:
def logAdd(para1, para2):
    """ Useful for adding magnitudes. """
    return np.log10((10.0 ** np.asarray(para1)) + 
                    (10.0 ** np.asarray(para2)))

def errAdd(err1, err2):
    """Add error quadral..."""
    return np.sqrt((err1 ** 2.0) + 
                   (err2 ** 2.0))

In [7]:
def toColorArr(data, bottom=None, top=None):
    """ 
    Convert a data array to "color array" (between 0 and 1). 
    
    Parameters:
        bottom, top  : 
    """
    if top is not None:
        data[data >= top] = top
    if bottom is not None:
        data[data <= bottom] = bottom
        
    return ((data - np.nanmin(data)) / 
            (np.nanmax(data) - np.nanmin(data))) * 255.0

In [12]:
def getLuminosity(mag, redshift, extinction=None, 
                  amag_sun=None):
    """Get the absolute magnitude or luminosity."""
    distmod = hUtil.cosmoDistMod(redshift)
    absMag = (mag - distmod)
    if extinction is not None: 
        absMag -= extinction 
    if amag_sun is not None: 
        absMag = ((amag_sun - absMag) / 2.5)
    
    return absMag

# Results by 2016-01-08

* Three major datasets: 
    
    1. redBCG - `/Users/songhuang/work/hscs/gama_massive/sbp/redbcg`: **219** galaxies (vetted)
        - `redmapper_bcg_hscmatch_mass_use_sbpsum_modA_muI1.fits`
    2. redMem - `/Users/songhuang/work/hscs/gama_massive/sbp/redmem`: **1578** galaxies (not vetted)
        - `redmapper_mem_hscmatch_mass_sbpsum_modA_muI1.fits`
    3. gama - `/Users/songhuang/work/hscs/gama_massive/sbp/gama`: **7646** galaxies (not vetted)
        - `gama_massive_160107_sbpsum_modA_muI1.fits`

In [8]:
newDir = '/Users/songhuang/work/hscs/gama_massive/sbp/'

# Folder for 3 datasets
bcgDir = os.path.join(newDir, 'redbcg')
memDir = os.path.join(newDir, 'redmem')
gamaDir = os.path.join(newDir, 'gama')

# Two summary catalogs
#bcgCat = os.path.join(bcgDir, 'redmapper_bcg_hscmatch_mass_use_sbpsum_modA_muI1.fits')
bcgCat = os.path.join(bcgDir, 'redbcg_mass_use.fits')
memCat = os.path.join(memDir, 'redmapper_mem_hscmatch_mass_sbpsum_modA_muI1.fits')
gamaCat = os.path.join(gamaDir, 'gama_massive_160107_sbpsum_modA_muI1.fits')

if not os.path.isfile(bcgCat):
    raise Exception("## Can not find catalog for BCGs : %s" % bcgCat)
else: 
    bcgTab = Table.read(bcgCat, format='fits')

if not os.path.isfile(memCat):
    raise Exception("## Can not find catalog for cluster members : %s" % memCat)
else: 
    memTab = Table.read(memCat, format='fits')
    
if not os.path.isfile(gamaCat):
    raise Exception("## Can not find catalog for GAMA galaxies : %s" % gamaCat)
else: 
    gamaTab = Table.read(gamaCat, format='fits')

In [9]:
print("## Deal with %i galaxies in BCG sample" % len(bcgTab))
print("## Deal with %i galaxies in Cluster member sample" % len(memTab))
print("## Deal with %i galaxies in GAMA sample" % len(gamaTab))

## Deal with 219 galaxies in BCG sample
## Deal with 1670 galaxies in Cluster member sample
## Deal with 8165 galaxies in GAMA sample


In [11]:
print(bcgTab.colnames)

['ID_CLUSTER', 'ID_USE', 'NAME', 'RA_BCG', 'DEC_BCG', 'Z_LAMBDA', 'Z_LAMBDA_ERR', 'LAMBDA_CLUSTER', 'LAMBDA_ERR_CLUSTER', 'S_CLUSTER', 'Z_SPEC_BCG', 'OBJID_BCG', 'IMAG_BCG', 'IMAG_ERR_BCG', 'MODEL_MAG_u_BCG', 'MODEL_MAG_g_BCG', 'MODEL_MAG_r_BCG', 'MODEL_MAG_i_BCG', 'MODEL_MAG_z_BCG', 'MODEL_MAGERR_u_BCG', 'MODEL_MAGERR_g_BCG', 'MODEL_MAGERR_r_BCG', 'MODEL_MAGERR_i_BCG', 'MODEL_MAGERR_z_BCG', 'ILUM_BCG', 'PZBINS_1', 'PZBINS_2', 'PZBINS_3', 'PZ_1', 'PZ_2', 'PZ_3', 'P_CEN_1', 'P_CEN_2', 'P_CEN_3', 'P_CEN_4', 'P_CEN_5', 'RA_CEN_1', 'RA_CEN_2', 'RA_CEN_3', 'RA_CEN_4', 'RA_CEN_5', 'DEC_CEN_1', 'DEC_CEN_2', 'DEC_CEN_3', 'DEC_CEN_4', 'DEC_CEN_5', 'ID_CEN_1', 'ID_CEN_2', 'ID_CEN_3', 'ID_CEN_4', 'ID_CEN_5', 'ra2000', 'decl2000', 'tract', 'patch', 'id', 'parent', 'a_g', 'a_r', 'a_i', 'a_z', 'a_y', 'gmag_psf', 'gmag_psf_err', 'rmag_psf', 'rmag_psf_err', 'imag_psf', 'imag_psf_err', 'zmag_psf', 'zmag_psf_err', 'ymag_psf', 'ymag_psf_err', 'gmag_kron', 'gmag_kron_err', 'rmag_kron', 'rmag_kron_err', 'i

# Estimate the luminosity, mass-to-light ratio, and stellar mass 
    - Consider the the K-correction 
    - Consider the differen mass models

In [18]:
## BCG 

## Luminosity based on hscPipe cModel: 
lumI_bcg = getLuminosity(bcgTab['imag_cmodel'], bcgTab['Z'], extinction=bcgTab['a_i'], 
                         amag_sun=amag_sun_des_i)
lumG_bcg = getLuminosity(bcgTab['gmag_cmodel'], bcgTab['Z'], extinction=bcgTab['a_g'], 
                         amag_sun=amag_sun_des_g)
lumR_bcg = getLuminosity(bcgTab['rmag_cmodel'], bcgTab['Z'], extinction=bcgTab['a_r'], 
                         amag_sun=amag_sun_des_r)
lumZ_bcg = getLuminosity(bcgTab['zmag_cmodel'], bcgTab['Z'], extinction=bcgTab['a_z'], 
                         amag_sun=amag_sun_des_z)

## K-corrected luminosity: 
lumI_kA_bcg = lumI_bcg - (bcgTab['KCORRECT_I'] / 2.5)
lumI_kB_bcg = lumI_bcg - (bcgTab['KCORRECT_b_I'] / 2.5)
lumI_kC_bcg = lumI_bcg - (bcgTab['KCORRECT_c_I'] / 2.5)

## K-corrected cModel color: 
### Model A
gr_kA_bcg = ((bcgTab['gmag_cmodel'] - bcgTab['a_g'] + bcgTab['KCORRECT_G']) - 
             (bcgTab['rmag_cmodel'] - bcgTab['a_r'] + bcgTab['KCORRECT_R']))
gi_kA_bcg = ((bcgTab['gmag_cmodel'] - bcgTab['a_g'] + bcgTab['KCORRECT_G']) - 
             (bcgTab['imag_cmodel'] - bcgTab['a_i'] + bcgTab['KCORRECT_I']))
gz_kA_bcg = ((bcgTab['gmag_cmodel'] - bcgTab['a_g'] + bcgTab['KCORRECT_G']) - 
             (bcgTab['zmag_cmodel'] - bcgTab['a_z'] + bcgTab['KCORRECT_Z']))
ri_kA_bcg = ((bcgTab['rmag_cmodel'] - bcgTab['a_r'] + bcgTab['KCORRECT_R']) - 
             (bcgTab['imag_cmodel'] - bcgTab['a_i'] + bcgTab['KCORRECT_I']))
### Model B
gr_kB_bcg = ((bcgTab['gmag_cmodel'] - bcgTab['a_g'] + bcgTab['KCORRECT_b_G']) - 
             (bcgTab['rmag_cmodel'] - bcgTab['a_r'] + bcgTab['KCORRECT_b_R']))
gi_kB_bcg = ((bcgTab['gmag_cmodel'] - bcgTab['a_g'] + bcgTab['KCORRECT_b_G']) - 
             (bcgTab['imag_cmodel'] - bcgTab['a_i'] + bcgTab['KCORRECT_b_I']))
gz_kB_bcg = ((bcgTab['gmag_cmodel'] - bcgTab['a_g'] + bcgTab['KCORRECT_b_G']) - 
             (bcgTab['zmag_cmodel'] - bcgTab['a_z'] + bcgTab['KCORRECT_b_Z']))
ri_kB_bcg = ((bcgTab['rmag_cmodel'] - bcgTab['a_r'] + bcgTab['KCORRECT_b_R']) - 
             (bcgTab['imag_cmodel'] - bcgTab['a_i'] + bcgTab['KCORRECT_b_I']))
### Model C
gr_kC_bcg = ((bcgTab['gmag_cmodel'] - bcgTab['a_g'] + bcgTab['KCORRECT_c_G']) - 
             (bcgTab['rmag_cmodel'] - bcgTab['a_r'] + bcgTab['KCORRECT_c_R']))
gi_kC_bcg = ((bcgTab['gmag_cmodel'] - bcgTab['a_g'] + bcgTab['KCORRECT_c_G']) - 
             (bcgTab['imag_cmodel'] - bcgTab['a_i'] + bcgTab['KCORRECT_c_I']))
gz_kC_bcg = ((bcgTab['gmag_cmodel'] - bcgTab['a_g'] + bcgTab['KCORRECT_c_G']) - 
             (bcgTab['zmag_cmodel'] - bcgTab['a_z'] + bcgTab['KCORRECT_c_Z']))
ri_kC_bcg = ((bcgTab['rmag_cmodel'] - bcgTab['a_r'] + bcgTab['KCORRECT_c_R']) - 
             (bcgTab['imag_cmodel'] - bcgTab['a_i'] + bcgTab['KCORRECT_c_I']))
    
## Stellar mass from iSEDFit 
logm2lI_A_bcg = (bcgTab['MSTAR'] - lumI_bcg)
logm2lI_B_bcg = (bcgTab['MSTAR_b'] - lumI_bcg)
logm2lI_C_bcg = (bcgTab['MSTAR_c'] - lumI_bcg)

In [19]:
bcgTab.add_column(Column(name='lumI_cmodel', data=lumI_bcg))
bcgTab.add_column(Column(name='lumG_cmodel', data=lumG_bcg))
bcgTab.add_column(Column(name='lumR_cmodel', data=lumR_bcg))
bcgTab.add_column(Column(name='lumZ_cmodel', data=lumZ_bcg))
bcgTab.add_column(Column(name='gr_kA', data=gr_kA_bcg))
bcgTab.add_column(Column(name='gi_kA', data=gi_kA_bcg))
bcgTab.add_column(Column(name='gz_kA', data=gz_kA_bcg))
bcgTab.add_column(Column(name='ri_kA', data=ri_kA_bcg))
bcgTab.add_column(Column(name='gr_kB', data=gr_kB_bcg))
bcgTab.add_column(Column(name='gi_kB', data=gi_kB_bcg))
bcgTab.add_column(Column(name='gz_kB', data=gz_kB_bcg))
bcgTab.add_column(Column(name='ri_kB', data=ri_kB_bcg))
bcgTab.add_column(Column(name='gr_kC', data=gr_kC_bcg))
bcgTab.add_column(Column(name='gi_kC', data=gi_kC_bcg))
bcgTab.add_column(Column(name='gz_kC', data=gz_kC_bcg))
bcgTab.add_column(Column(name='ri_kC', data=ri_kC_bcg))
bcgTab.add_column(Column(name='logm2lI_A', data=logm2lI_A_bcg))
bcgTab.add_column(Column(name='logm2lI_B', data=logm2lI_B_bcg))
bcgTab.add_column(Column(name='logm2lI_C', data=logm2lI_C_bcg))

In [20]:
## Mem

## Luminosity based on hscPipe cModel: 
lumI_mem = getLuminosity(memTab['imag_cmodel'], memTab['Z'], extinction=memTab['a_i'], 
                         amag_sun=amag_sun_des_i)
lumG_mem = getLuminosity(memTab['gmag_cmodel'], memTab['Z'], extinction=memTab['a_g'], 
                         amag_sun=amag_sun_des_g)
lumR_mem = getLuminosity(memTab['rmag_cmodel'], memTab['Z'], extinction=memTab['a_r'], 
                         amag_sun=amag_sun_des_r)
lumZ_mem = getLuminosity(memTab['zmag_cmodel'], memTab['Z'], extinction=memTab['a_z'], 
                         amag_sun=amag_sun_des_z)

## K-corrected luminosity: 
lumI_kA_mem = lumI_mem - (memTab['KCORRECT_I'] / 2.5)
lumI_kB_mem = lumI_mem - (memTab['KCORRECT_b_I'] / 2.5)
lumI_kC_mem = lumI_mem - (memTab['KCORRECT_c_I'] / 2.5)

## K-corrected cModel color: 
### Model A
gr_kA_mem = ((memTab['gmag_cmodel'] - memTab['a_g'] + memTab['KCORRECT_G']) - 
             (memTab['rmag_cmodel'] - memTab['a_r'] + memTab['KCORRECT_R']))
gi_kA_mem = ((memTab['gmag_cmodel'] - memTab['a_g'] + memTab['KCORRECT_G']) - 
             (memTab['imag_cmodel'] - memTab['a_i'] + memTab['KCORRECT_I']))
gz_kA_mem = ((memTab['gmag_cmodel'] - memTab['a_g'] + memTab['KCORRECT_G']) - 
             (memTab['zmag_cmodel'] - memTab['a_z'] + memTab['KCORRECT_Z']))
ri_kA_mem = ((memTab['rmag_cmodel'] - memTab['a_r'] + memTab['KCORRECT_R']) - 
             (memTab['imag_cmodel'] - memTab['a_i'] + memTab['KCORRECT_I']))
### Model B
gr_kB_mem = ((memTab['gmag_cmodel'] - memTab['a_g'] + memTab['KCORRECT_b_G']) - 
             (memTab['rmag_cmodel'] - memTab['a_r'] + memTab['KCORRECT_b_R']))
gi_kB_mem = ((memTab['gmag_cmodel'] - memTab['a_g'] + memTab['KCORRECT_b_G']) - 
             (memTab['imag_cmodel'] - memTab['a_i'] + memTab['KCORRECT_b_I']))
gz_kB_mem = ((memTab['gmag_cmodel'] - memTab['a_g'] + memTab['KCORRECT_b_G']) - 
             (memTab['zmag_cmodel'] - memTab['a_z'] + memTab['KCORRECT_b_Z']))
ri_kB_mem = ((memTab['rmag_cmodel'] - memTab['a_r'] + memTab['KCORRECT_b_R']) - 
             (memTab['imag_cmodel'] - memTab['a_i'] + memTab['KCORRECT_b_I']))
### Model C
gr_kC_mem = ((memTab['gmag_cmodel'] - memTab['a_g'] + memTab['KCORRECT_c_G']) - 
             (memTab['rmag_cmodel'] - memTab['a_r'] + memTab['KCORRECT_c_R']))
gi_kC_mem = ((memTab['gmag_cmodel'] - memTab['a_g'] + memTab['KCORRECT_c_G']) - 
             (memTab['imag_cmodel'] - memTab['a_i'] + memTab['KCORRECT_c_I']))
gz_kC_mem = ((memTab['gmag_cmodel'] - memTab['a_g'] + memTab['KCORRECT_c_G']) - 
             (memTab['zmag_cmodel'] - memTab['a_z'] + memTab['KCORRECT_c_Z']))
ri_kC_mem = ((memTab['rmag_cmodel'] - memTab['a_r'] + memTab['KCORRECT_c_R']) - 
             (memTab['imag_cmodel'] - memTab['a_i'] + memTab['KCORRECT_c_I']))
    
## Stellar mass from iSEDFit 
logm2lI_A_mem = (memTab['MSTAR'] - lumI_mem)
logm2lI_B_mem = (memTab['MSTAR_b'] - lumI_mem)
logm2lI_C_mem = (memTab['MSTAR_c'] - lumI_mem)

In [22]:
memTab.add_column(Column(name='lumI_cmodel', data=lumI_mem))
memTab.add_column(Column(name='lumG_cmodel', data=lumG_mem))
memTab.add_column(Column(name='lumR_cmodel', data=lumR_mem))
memTab.add_column(Column(name='lumZ_cmodel', data=lumZ_mem))
memTab.add_column(Column(name='gr_kA', data=gr_kA_mem))
memTab.add_column(Column(name='gi_kA', data=gi_kA_mem))
memTab.add_column(Column(name='gz_kA', data=gz_kA_mem))
memTab.add_column(Column(name='ri_kA', data=ri_kA_mem))
memTab.add_column(Column(name='gr_kB', data=gr_kB_mem))
memTab.add_column(Column(name='gi_kB', data=gi_kB_mem))
memTab.add_column(Column(name='gz_kB', data=gz_kB_mem))
memTab.add_column(Column(name='ri_kB', data=ri_kB_mem))
memTab.add_column(Column(name='gr_kC', data=gr_kC_mem))
memTab.add_column(Column(name='gi_kC', data=gi_kC_mem))
memTab.add_column(Column(name='gz_kC', data=gz_kC_mem))
memTab.add_column(Column(name='ri_kC', data=ri_kC_mem))
memTab.add_column(Column(name='logm2lI_A', data=logm2lI_A_mem))
memTab.add_column(Column(name='logm2lI_B', data=logm2lI_B_mem))
memTab.add_column(Column(name='logm2lI_C', data=logm2lI_C_mem))

In [21]:
## GAMA

## Luminosity based on hscPipe cModel: 
lumI_gama = getLuminosity(gamaTab['imag_cmodel'], gamaTab['Z'], extinction=gamaTab['a_i'], 
                         amag_sun=amag_sun_des_i)
lumG_gama = getLuminosity(gamaTab['gmag_cmodel'], gamaTab['Z'], extinction=gamaTab['a_g'], 
                         amag_sun=amag_sun_des_g)
lumR_gama = getLuminosity(gamaTab['rmag_cmodel'], gamaTab['Z'], extinction=gamaTab['a_r'], 
                         amag_sun=amag_sun_des_r)
lumZ_gama = getLuminosity(gamaTab['zmag_cmodel'], gamaTab['Z'], extinction=gamaTab['a_z'], 
                         amag_sun=amag_sun_des_z)

## K-corrected luminosity: 
lumI_kA_gama = lumI_gama - (gamaTab['KCORRECT_I'] / 2.5)
lumI_kB_gama = lumI_gama - (gamaTab['KCORRECT_b_I'] / 2.5)
lumI_kC_gama = lumI_gama - (gamaTab['KCORRECT_c_I'] / 2.5)

## K-corrected cModel color: 
### Model A
gr_kA_gama = ((gamaTab['gmag_cmodel'] - gamaTab['a_g'] + gamaTab['KCORRECT_G']) - 
             (gamaTab['rmag_cmodel'] - gamaTab['a_r'] + gamaTab['KCORRECT_R']))
gi_kA_gama = ((gamaTab['gmag_cmodel'] - gamaTab['a_g'] + gamaTab['KCORRECT_G']) - 
             (gamaTab['imag_cmodel'] - gamaTab['a_i'] + gamaTab['KCORRECT_I']))
gz_kA_gama = ((gamaTab['gmag_cmodel'] - gamaTab['a_g'] + gamaTab['KCORRECT_G']) - 
             (gamaTab['zmag_cmodel'] - gamaTab['a_z'] + gamaTab['KCORRECT_Z']))
ri_kA_gama = ((gamaTab['rmag_cmodel'] - gamaTab['a_r'] + gamaTab['KCORRECT_R']) - 
             (gamaTab['imag_cmodel'] - gamaTab['a_i'] + gamaTab['KCORRECT_I']))
### Model B
gr_kB_gama = ((gamaTab['gmag_cmodel'] - gamaTab['a_g'] + gamaTab['KCORRECT_b_G']) - 
             (gamaTab['rmag_cmodel'] - gamaTab['a_r'] + gamaTab['KCORRECT_b_R']))
gi_kB_gama = ((gamaTab['gmag_cmodel'] - gamaTab['a_g'] + gamaTab['KCORRECT_b_G']) - 
             (gamaTab['imag_cmodel'] - gamaTab['a_i'] + gamaTab['KCORRECT_b_I']))
gz_kB_gama = ((gamaTab['gmag_cmodel'] - gamaTab['a_g'] + gamaTab['KCORRECT_b_G']) - 
             (gamaTab['zmag_cmodel'] - gamaTab['a_z'] + gamaTab['KCORRECT_b_Z']))
ri_kB_gama = ((gamaTab['rmag_cmodel'] - gamaTab['a_r'] + gamaTab['KCORRECT_b_R']) - 
             (gamaTab['imag_cmodel'] - gamaTab['a_i'] + gamaTab['KCORRECT_b_I']))
### Model C
gr_kC_gama = ((gamaTab['gmag_cmodel'] - gamaTab['a_g'] + gamaTab['KCORRECT_c_G']) - 
             (gamaTab['rmag_cmodel'] - gamaTab['a_r'] + gamaTab['KCORRECT_c_R']))
gi_kC_gama = ((gamaTab['gmag_cmodel'] - gamaTab['a_g'] + gamaTab['KCORRECT_c_G']) - 
             (gamaTab['imag_cmodel'] - gamaTab['a_i'] + gamaTab['KCORRECT_c_I']))
gz_kC_gama = ((gamaTab['gmag_cmodel'] - gamaTab['a_g'] + gamaTab['KCORRECT_c_G']) - 
             (gamaTab['zmag_cmodel'] - gamaTab['a_z'] + gamaTab['KCORRECT_c_Z']))
ri_kC_gama = ((gamaTab['rmag_cmodel'] - gamaTab['a_r'] + gamaTab['KCORRECT_c_R']) - 
             (gamaTab['imag_cmodel'] - gamaTab['a_i'] + gamaTab['KCORRECT_c_I']))
    
## Stellar mass from iSEDFit 
logm2lI_A_gama = (gamaTab['MSTAR'] - lumI_gama)
logm2lI_B_gama = (gamaTab['MSTAR_b'] - lumI_gama)
logm2lI_C_gama = (gamaTab['MSTAR_c'] - lumI_gama)

In [23]:
gamaTab.add_column(Column(name='lumI_cmodel', data=lumI_gama))
gamaTab.add_column(Column(name='lumG_cmodel', data=lumG_gama))
gamaTab.add_column(Column(name='lumR_cmodel', data=lumR_gama))
gamaTab.add_column(Column(name='lumZ_cmodel', data=lumZ_gama))
gamaTab.add_column(Column(name='gr_kA', data=gr_kA_gama))
gamaTab.add_column(Column(name='gi_kA', data=gi_kA_gama))
gamaTab.add_column(Column(name='gz_kA', data=gz_kA_gama))
gamaTab.add_column(Column(name='ri_kA', data=ri_kA_gama))
gamaTab.add_column(Column(name='gr_kB', data=gr_kB_gama))
gamaTab.add_column(Column(name='gi_kB', data=gi_kB_gama))
gamaTab.add_column(Column(name='gz_kB', data=gz_kB_gama))
gamaTab.add_column(Column(name='ri_kB', data=ri_kB_gama))
gamaTab.add_column(Column(name='gr_kC', data=gr_kC_gama))
gamaTab.add_column(Column(name='gi_kC', data=gi_kC_gama))
gamaTab.add_column(Column(name='gz_kC', data=gz_kC_gama))
gamaTab.add_column(Column(name='ri_kC', data=ri_kC_gama))
gamaTab.add_column(Column(name='logm2lI_A', data=logm2lI_A_gama))
gamaTab.add_column(Column(name='logm2lI_B', data=logm2lI_B_gama))
gamaTab.add_column(Column(name='logm2lI_C', data=logm2lI_C_gama))

## Basic properties of the sample:
    * Redshift-Luminosity relation 
    * Redshift-Color relation 
    * Redshift-MStar relation
    * Mstar-color relation

## Compare the stellar mass estimates 

## SBP compare 

In [19]:
gama1 = gamaTab[(gamaTab['Z'] >= 0.20) & 
                ((gamaTab['lum_100'] + gamaTab['logm2l_i']) >= 11.4) & 
                ((gamaTab['lum_100'] + gamaTab['logm2l_i']) <= 11.6)]

mem1 = memTab[(memTab['Z'] >= 0.20) & 
              ((memTab['lum_100'] + memTab['logm2l_i']) >= 11.4) & 
              ((memTab['lum_100'] + memTab['logm2l_i']) <= 11.6)]

bcg1 = bcgTab[(bcgTab['Z'] >= 0.10) & 
              (bcgTab['Z'] <= 0.50) & 
              ((bcgTab['lum_100'] + bcgTab['logm2l_i']) >= 11.4) & 
              ((bcgTab['lum_100'] + bcgTab['logm2l_i']) <= 11.6) & 
              (bcgTab['P_CEN_1'] >= 0.8)]

print("## 0.2 < z < 0.4 ; 11.4 < logM* < 11.6 using SBP Mass \n")

print("## GAMA1 : %i galaxies" % len(gama1))
print("##     Median redshift  : ", np.nanmedian(gama1['Z']))
print("##     Median logm_sbp  : ", np.nanmedian(gama1['lum_100'] + gama1['logm2l_i']))
print("##     Median logm_gama : ", np.nanmedian(gama1['logms_gama']))

print("\n## MEM1  : %i galaxies" % len(mem1))
print("##     Median redshift  : ", np.nanmedian(mem1['Z']))
print("##     Median logm_sbp  : ", np.nanmedian(mem1['lum_100'] + mem1['logm2l_i']))
print("##     Median logm_gama : ", np.nanmedian(mem1['logms_gama']))

print("\n## BCG1  : %i galaxies" % len(bcg1))
print("##     Median redshift  : ", np.nanmedian(bcg1['Z']))
print("##     Median logm_sbp  : ", np.nanmedian(bcg1['lum_100'] + bcg1['logm2l_i']))
print("##     Median logm_gama : ", np.nanmedian(bcg1['logms_gama']))

## 0.2 < z < 0.4 ; 11.4 < logM* < 11.6 using SBP Mass 

## GAMA1 : 1350 galaxies
##     Median redshift  :  0.309364974499
##     Median logm_sbp  :  11.4801063424
##     Median logm_gama :  11.2821502908

## MEM1  : 281 galaxies
##     Median redshift  :  0.317588478327
##     Median logm_sbp  :  11.4813548094
##     Median logm_gama :  11.3219896485

## BCG1  : 38 galaxies
##     Median redshift  :  0.31767821312
##     Median logm_sbp  :  11.5174923883
##     Median logm_gama :  11.3245989129


In [20]:
confidence_interval(bcg1['Z'], alpha=np.asarray([0.3173, 1.0]), 
                    metric=np.median, numResamples=1000, interpolate=True)

confidence_interval(mem1['Z'], alpha=np.asarray([0.3173, 1.0]), 
                    metric=np.median, numResamples=1000, interpolate=True)

confidence_interval(gama1['Z'], alpha=np.asarray([0.3173, 1.0]), 
                    metric=np.median, numResamples=1000, interpolate=True)

array([ 0.30524999,  0.3122051 ,  0.30936497,  0.30936497])